In [24]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [25]:
pd.set_option('display.max_columns', None)

In [26]:
df = pd.read_csv("data/cleaned_data/hyderabad/hyd_properties_imputed.csv")
df.head(3)

,property_type,bhk,area,locality,city,price_in_crore,price_per_sqft,new_resale,age_of_property,bathrooms,carpet_area,floor,balconies,additional_rooms,furnishing_type,facing,amenities,furnish_details
0,Apartment,2,1120.0,Asmangarh,Hyderabad,0.58,5178.0,Resale,Old Property,2,993.55,4.0,1.0,pooja room,Semi-Furnished,North,"Landscaped Gardens, Lift(s), Swimming Pool, Gy...",Sofa
1,Apartment,3,1905.0,Kokapet,Hyderabad,1.39,7349.0,New,Unknown,3,1689.92,9.0,1.0,NaN,Unfurnished,East,NaN,NaN
2,Independent House,10,14400.0,Banjara Hills,Hyderabad,17.47,12131.0,Resale,Unknown,11,12774.19,2.0,1.0,NaN,Semi-Furnished,North,NaN,NaN


In [27]:
df.shape

(13565, 18)

#### Check Duplicate Rows

In [28]:
df.duplicated().sum()

np.int64(113)

In [29]:
df[df.duplicated(keep=False)].sort_values(['property_type','bhk','area','locality'])

,property_type,bhk,area,locality,city,price_in_crore,price_per_sqft,new_resale,age_of_property,bathrooms,carpet_area,floor,balconies,additional_rooms,furnishing_type,facing,amenities,furnish_details
5724,Apartment,2,1034.0,Attapur,Hyderabad,0.62,5996.0,New,New Property,2,917.26,2.0,1.0,NaN,Unfurnished,East,"Car Parking, Childrens play area, Full Power B...",NaN
10491,Apartment,2,1034.0,Attapur,Hyderabad,0.62,5996.0,New,New Property,2,917.26,2.0,1.0,NaN,Unfurnished,East,"Car Parking, Childrens play area, Full Power B...",NaN
4873,Apartment,2,1060.0,Hyder Nagar,Hyderabad,0.38,3584.0,New,Unknown,2,940.32,4.0,2.0,NaN,Unfurnished,East,NaN,NaN
6550,Apartment,2,1060.0,Hyder Nagar,Hyderabad,0.38,3584.0,New,Unknown,2,940.32,4.0,2.0,NaN,Unfurnished,East,NaN,NaN
918,Apartment,2,1065.0,Vanasthalipuram,Hyderabad,0.55,5164.0,New,Unknown,2,944.76,1.0,1.0,NaN,Unfurnished,East,"Full Power Backup, Lift(s)",Microwave
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9988,Villa,3,1830.0,Patancheru,Hyderabad,1.18,6448.0,New,Unknown,3,1623.39,2.0,1.0,NaN,Unfurnished,East,NaN,NaN
955,Villa,3,2000.0,Bahadurpally,Hyderabad,1.20,6000.0,Resale,Unknown,3,1774.19,2.0,1.0,NaN,Unfurnished,East,"Jogging Track, Indoor Games, Swimming Pool, Fu...",NaN
4034,Villa,3,2000.0,Bahadurpally,Hyderabad,1.20,6000.0,Resale,Unknown,3,1774.19,2.0,1.0,NaN,Unfurnished,East,"Jogging Track, Indoor Games, Swimming Pool, Fu...",NaN
2181,Villa,3,2636.0,Patancheru,Hyderabad,1.76,6676.0,New,Unknown,3,2338.39,2.0,1.0,NaN,Unfurnished,East,NaN,NaN


In [30]:
df1 = df.drop_duplicates()
df1.duplicated().sum()

np.int64(0)

In [31]:
df1.isnull().sum()

property_type           0
bhk                     0
area                    0
locality                0
city                    0
price_in_crore          0
price_per_sqft          0
new_resale              0
age_of_property         0
bathrooms               0
carpet_area             0
floor                   0
balconies               0
additional_rooms    11301
furnishing_type         0
facing                  0
amenities            3319
furnish_details      9317
dtype: int64

In [32]:
df1.columns

Index(['property_type', 'bhk', 'area', 'locality', 'city', 'price_in_crore',
       'price_per_sqft', 'new_resale', 'age_of_property', 'bathrooms',
       'carpet_area', 'floor', 'balconies', 'additional_rooms',
       'furnishing_type', 'facing', 'amenities', 'furnish_details'],
      dtype='object')

<h2 align='center' style="color:blue">Feature Engineering</h2>

#### Create a amenities_score based on amenities column

In [33]:
def get_all_unique_amenities(amenities_series):
    all_amenities = amenities_series.dropna().str.cat(sep=', ')
    unique_amenities = set(amenity.strip() for amenity in all_amenities.split(', '))
    return unique_amenities
unique_amenities = get_all_unique_amenities(df1['amenities'])
unique_amenities

{'24 X 7 Security',
 'ATM',
 'Cafeteria',
 'Car Parking',
 'Childrens play area',
 'Club House',
 'Full Power Backup',
 'Golf Course',
 'Gymnasium',
 'Hospital',
 'Indoor Games',
 'Intercom',
 'Jogging Track',
 'Landscaped Gardens',
 'Lift(s)',
 'Maintenance Staff',
 'Multipurpose Room',
 'Rain Water Harvesting',
 'School',
 'Shopping Mall',
 'Sports Facility',
 'Staff Quarter',
 'Swimming Pool',
 'Vaastu Compliant'}

In [34]:
# Define scores for each amenity
amenity_scores = {
    '24 X 7 Security': 5,
    'ATM': 3,
    'Cafeteria': 3,
    'Car Parking': 4,
    'Childrens play area': 4,
    'Club House': 4,
    'Full Power Backup': 5,
    'Golf Course': 5,
    'Gymnasium': 4,
    'Hospital': 5,
    'Indoor Games': 3,
    'Intercom': 2,
    'Jogging Track': 4,
    'Landscaped Gardens': 4,
    'Lift(s)': 4,
    'Maintenance Staff': 3,
    'Multipurpose Room': 4,
    'Rain Water Harvesting': 3,
    'School': 5,
    'Shopping Mall': 5,
    'Sports Facility': 4,
    'Staff Quarter': 2,
    'Swimming Pool': 5,
    'Vaastu Compliant': 3
}


def score_amenities(amenities_str):
    if pd.isna(amenities_str):
        return 0  
    amenities_list = [amenity.strip() for amenity in amenities_str.split(',')]
    return sum(amenity_scores.get(amenity, 0) for amenity in amenities_list)

df1['amenities_score'] = df1['amenities'].apply(score_amenities)
df1[['amenities', 'amenities_score']].head()

,amenities,amenities_score
0,"Landscaped Gardens, Lift(s), Swimming Pool, Gy...",31
1,NaN,0
2,NaN,0
3,"Gymnasium, Lift(s), Intercom, Sports Facility,...",32
4,NaN,0


#### Create furnishing_score based on furnish_details columnm

In [35]:
def get_all_unique_furnishings(furnish_series):
    all_furnishings = furnish_series.dropna().str.cat(sep=', ')
    unique_furnishings = set(furnishing.strip() for furnishing in all_furnishings.split(', '))
    return unique_furnishings
unique_furnishings = get_all_unique_furnishings(df1['furnish_details'])
unique_furnishings

{'AC',
 'BED',
 'Dining Table',
 'Gas connection',
 'Microwave',
 'Refrigerator',
 'Sofa',
 'TV',
 'Wardrobe',
 'Washing Machine',
 'Wifi'}

In [36]:
# Define scores for each furnish_type
furnish_details_scores = {
    'AC': 8,
    'BED': 10,
    'Dining Table': 7,
    'Gas connection': 6,
    'Microwave': 7,
    'Refrigerator': 8,
    'Sofa': 6,
    'TV': 5,
    'Wardrobe': 7,
    'Washing Machine': 8,
    'Wifi': 9
}

def score_furnishings(furnishing_str):
    if pd.isna(furnishing_str):
        return 0  
    furnishings_list = [furnishing.strip() for furnishing in furnishing_str.split(',')]
    return sum(furnish_details_scores.get(furnishing, 0) for furnishing in furnishings_list)

df1['furnishing_score'] = df1['furnish_details'].apply(score_furnishings)
df1[~df1['furnish_details'].isnull()][['furnish_details','furnishing_score']].head()

,furnish_details,furnishing_score
0,Sofa,6
3,Gas connection,6
5,Sofa,6
6,"BED, Dining Table, Sofa, TV, Wardrobe",35
8,"BED, Wifi",19


#### Create new columns based on the presence of the respective additional_rooms  

In [37]:
df1['study_room'] = df1['additional_rooms'].apply(lambda x: 1 if pd.notna(x) and 'study room' in x else 0)
df1['servant_room'] = df1['additional_rooms'].apply(lambda x: 1 if pd.notna(x) and 'servant room' in x else 0)
df1['pooja_room'] = df1['additional_rooms'].apply(lambda x: 1 if pd.notna(x) and 'pooja room' in x else 0)

In [38]:
df1.head(3)

,property_type,bhk,area,locality,city,price_in_crore,price_per_sqft,new_resale,age_of_property,bathrooms,carpet_area,floor,balconies,additional_rooms,furnishing_type,facing,amenities,furnish_details,amenities_score,furnishing_score,study_room,servant_room,pooja_room
0,Apartment,2,1120.0,Asmangarh,Hyderabad,0.58,5178.0,Resale,Old Property,2,993.55,4.0,1.0,pooja room,Semi-Furnished,North,"Landscaped Gardens, Lift(s), Swimming Pool, Gy...",Sofa,31,6,0,0,1
1,Apartment,3,1905.0,Kokapet,Hyderabad,1.39,7349.0,New,Unknown,3,1689.92,9.0,1.0,NaN,Unfurnished,East,NaN,NaN,0,0,0,0,0
2,Independent House,10,14400.0,Banjara Hills,Hyderabad,17.47,12131.0,Resale,Unknown,11,12774.19,2.0,1.0,NaN,Semi-Furnished,North,NaN,NaN,0,0,0,0,0


In [16]:
# def get_all_unique_rooms(room_series):
#     all_rooms = room_series.dropna().str.cat(sep=', ')
#     unique_rooms = set(room.strip() for room in all_rooms.split(', '))
#     return unique_rooms
# unique_rooms = get_all_unique_rooms(df1['additional_rooms'])
# unique_rooms

In [17]:
# additional_rooms_scores = {
#     'pooja room': 3,
#     'study room': 2,
#     'servant room': 2,
# }

# def score_additional_rooms(rooms):
#     if pd.isna(rooms):
#         return 0
#     room_list = [room.strip() for room in rooms.split(',')]
#     return sum(additional_rooms_scores.get(room, 0) for room in room_list)
    
# df1['additional_rooms_score'] = df1['additional_rooms'].apply(score_additional_rooms)
# df1[~df1['additional_rooms'].isnull()][['additional_rooms','additional_rooms_score']].sample(5)

#### Treat the age_of_property column

In [39]:
df1.groupby('new_resale')['age_of_property'].value_counts()

new_resale  age_of_property        
New         Unknown                    8233
            New Property               1493
            Moderately New Property     533
            Old Property                 14
            Very Old Property             1
Resale      Unknown                    1239
            Moderately New Property     931
            New Property                529
            Old Property                477
            Very Old Property             2
Name: count, dtype: int64

- In this approach, missing or "Unknown" values in the age_of_property column are handled as follows:
    - For New properties, "Unknown" values are replaced with "New Property", indicating the property is newly built.
    - For Resale properties, "Unknown" values are replaced with "Moderately New Property", representing the most frequent category for resale properties.

In [40]:
# For "New" properties, set "Unknown" to "New Property"
df1.loc[(df1['new_resale'] == 'New') & (df1['age_of_property'] == 'Unknown'), 'age_of_property'] = 'New Property'

# Assign the most frequent value to "Moderately New Property" resale properties
df1.loc[(df1['new_resale'] == 'Resale') & (df1['age_of_property'] == 'Unknown'), 'age_of_property'] = 'Moderately New Property'

df1['age_of_property'].value_counts()

age_of_property
New Property               10255
Moderately New Property     2703
Old Property                 491
Very Old Property              3
Name: count, dtype: int64

In [41]:
df1.head(3)

,property_type,bhk,area,locality,city,price_in_crore,price_per_sqft,new_resale,age_of_property,bathrooms,carpet_area,floor,balconies,additional_rooms,furnishing_type,facing,amenities,furnish_details,amenities_score,furnishing_score,study_room,servant_room,pooja_room
0,Apartment,2,1120.0,Asmangarh,Hyderabad,0.58,5178.0,Resale,Old Property,2,993.55,4.0,1.0,pooja room,Semi-Furnished,North,"Landscaped Gardens, Lift(s), Swimming Pool, Gy...",Sofa,31,6,0,0,1
1,Apartment,3,1905.0,Kokapet,Hyderabad,1.39,7349.0,New,New Property,3,1689.92,9.0,1.0,NaN,Unfurnished,East,NaN,NaN,0,0,0,0,0
2,Independent House,10,14400.0,Banjara Hills,Hyderabad,17.47,12131.0,Resale,Moderately New Property,11,12774.19,2.0,1.0,NaN,Semi-Furnished,North,NaN,NaN,0,0,0,0,0


In [42]:
df1['property_type'].unique()

array(['Apartment', 'Independent House', 'Villa', 'Independent Floor'],
      dtype=object)

- In propert_type column the values are 'Apartment', 'Independent House', 'Villa', 'Independent Floor'.
- Mapping Categories:
  - For example, "Villa" and "Independent House" might be grouped under a broader category like "Houses", and "Apartment" and "Independent Floor" might be grouped under "Flats".

In [43]:
property_map = {
    'Apartment': 'Flat',
    'Independent Floor': 'Flat',
    'Villa': 'House',
    'Independent House': 'House'
}

df1['property_type'] =  df1['property_type'].map(property_map)
df1['property_type'].unique()

array(['Flat', 'House'], dtype=object)

In [44]:
df1.head(3)

,property_type,bhk,area,locality,city,price_in_crore,price_per_sqft,new_resale,age_of_property,bathrooms,carpet_area,floor,balconies,additional_rooms,furnishing_type,facing,amenities,furnish_details,amenities_score,furnishing_score,study_room,servant_room,pooja_room
0,Flat,2,1120.0,Asmangarh,Hyderabad,0.58,5178.0,Resale,Old Property,2,993.55,4.0,1.0,pooja room,Semi-Furnished,North,"Landscaped Gardens, Lift(s), Swimming Pool, Gy...",Sofa,31,6,0,0,1
1,Flat,3,1905.0,Kokapet,Hyderabad,1.39,7349.0,New,New Property,3,1689.92,9.0,1.0,NaN,Unfurnished,East,NaN,NaN,0,0,0,0,0
2,House,10,14400.0,Banjara Hills,Hyderabad,17.47,12131.0,Resale,Moderately New Property,11,12774.19,2.0,1.0,NaN,Semi-Furnished,North,NaN,NaN,0,0,0,0,0


#### Drop the columns which are not required

In [46]:
cols_to_drop = ['price_per_sqft','additional_rooms','amenities','furnish_details']
df2 = df1.drop(cols_to_drop, axis='columns')
df2.head(3)

,property_type,bhk,area,locality,city,price_in_crore,new_resale,age_of_property,bathrooms,carpet_area,floor,balconies,furnishing_type,facing,amenities_score,furnishing_score,study_room,servant_room,pooja_room
0,Flat,2,1120.0,Asmangarh,Hyderabad,0.58,Resale,Old Property,2,993.55,4.0,1.0,Semi-Furnished,North,31,6,0,0,1
1,Flat,3,1905.0,Kokapet,Hyderabad,1.39,New,New Property,3,1689.92,9.0,1.0,Unfurnished,East,0,0,0,0,0
2,House,10,14400.0,Banjara Hills,Hyderabad,17.47,Resale,Moderately New Property,11,12774.19,2.0,1.0,Semi-Furnished,North,0,0,0,0,0


#### save this imputed df to csv

In [47]:
df2.to_csv("data/cleaned_data/hyderabad/hyd_properties_feature_engineered.csv", index=False)